In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [13]:
!pip install tensorflow==2.13.0 tensorflow-hub

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.2/524.2 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.17.0
    Uninstalling tf_keras-2.17.0:
      Successfully uninstalled tf_keras-2.17.0
  Attempting uninstal

In [3]:
#transfer learning with tensor flow part one
#transfer learning leverageing an exisiting models architeture and existing patterns for our own benefit..
#then we can adapt those paattenrs to our own data..

#download anda become one with the data

#swtitch to a gpu

#are we using a gpu>

donwnloading and becoming one with the data..

In [2]:
import zipfile
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2025-01-21 15:12:16--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 142.250.141.207, 74.125.137.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   131MB/s    in 1.2s    

2025-01-21 15:12:18 (131 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [3]:
#how many images in each folder
import os

In [4]:
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories, {len(filenames)} images in '{dirpath}'")

#the test data is the same the only differnece is instead of 750 images in each class we now how 75 examples.. we have 10 times less examples for our model to learn..
#intuitavely less data means worse results and less opportunity for our model to learn

There are 2 directories, 0 images in '10_food_classes_10_percent'
There are 10 directories, 0 images in '10_food_classes_10_percent/test'
There are 0 directories, 250 images in '10_food_classes_10_percent/test/ice_cream'
There are 0 directories, 250 images in '10_food_classes_10_percent/test/chicken_wings'
There are 0 directories, 250 images in '10_food_classes_10_percent/test/sushi'
There are 0 directories, 250 images in '10_food_classes_10_percent/test/hamburger'
There are 0 directories, 250 images in '10_food_classes_10_percent/test/chicken_curry'
There are 0 directories, 250 images in '10_food_classes_10_percent/test/grilled_salmon'
There are 0 directories, 250 images in '10_food_classes_10_percent/test/fried_rice'
There are 0 directories, 250 images in '10_food_classes_10_percent/test/steak'
There are 0 directories, 250 images in '10_food_classes_10_percent/test/pizza'
There are 0 directories, 250 images in '10_food_classes_10_percent/test/ramen'
There are 10 directories, 0 images

In [5]:
#preparing the data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_SHAPE = (224,224)
BATCH_SIZE = 32
EPOCHS = 5

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

print("training images: ")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size = IMG_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode = "categorical")

print("testing images: ")
test_data_10_percent = test_datagen.flow_from_directory(test_dir,
                                                        target_size = IMG_SHAPE,
                                                        batch_size = BATCH_SIZE,
                                                        class_mode = "categorical")

training images: 
Found 750 images belonging to 10 classes.
testing images: 
Found 2500 images belonging to 10 classes.


In [6]:
#setting up callbacks things to run while our model trains extra funcitonality you can add to your models to be perfromed during or after training
#some of the most popular call backs tracking experiments with the tensorboard call back, stopping a model from training befire it trains too long and overfits with the early stopping call back

#tensorflow callbacks

#in the past we tracked experiments by scolling up and down the notebook and checking the val accuracy.. but tensor board call back helps us to fix this.

import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f"saving tensorboard log files to {log_dir}")
  return tensorboard_callback

In [ ]:
#in the past we used tensorflow to create our own models layer by layer from scratch however now the majority of our layers will come  from tf.hub.dev

#browsing the tensorflow hub page and sorting for feature classification we found the folling feature hub link.. 10 mins exploring the tensorflow hub off a whim

#going to pit two modes against each other..
#lets compare the following two models



In [7]:
efficientnet_url = "https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet1k-b0-classification/versions/2"

resnet_url = "https://www.kaggle.com/models/tensorflow/resnet-50/TensorFlow2/feature-vector/1"


In [8]:
#tensorflow makes using a pretrained model as simple as calling a url

#going to need a fwe dependencies

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense

def create_model(model_url, numclasses = 10):
  """
  takes a tensorflow hub url and creates a keras sequential model with it:
    Args:
      model_url (str) : A tensorflow hub feature extraction url
      num_classes (int) : num of output neurons in the output layer.. should be equal to target classes defualt is 10

    returns:
      uncompiled keras sequential model with model_url as feature extractor layer and dense output layer with num_classes output neurons
  """
  #download the pretrained model and save it as a keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable = False, #freeze the already learned patterns
                                           name = "feature_extraction_layer",
                                           input_shape = IMG_SHAPE + (3, ))


  #create our own model
  model = tf.keras.Sequential([
      feature_extractor_layer,
      tf.keras.layers.Dense(10, activation = "softmax", name = "output_layer")
  ])

  return model


In [10]:
resnet_model = create_model(resnet_url, 10)

In [ ]:
#wow i couldnt figure out how to solve the issues with why i couldnt get this model to run but
#i realized it was because one of the packages was not installed properly.

In [11]:
resnet_model.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)



In [12]:
history = resnet_model.fit(train_data_10_percent,
                           epochs = 5,
                           steps_per_epoch = len(train_data_10_percent),
                           validation_data = test_data_10_percent,
                           validation_steps = len(test_data_10_percent),
                           callbacks = [create_tensorboard_callback(dir_name = "tensorflow hub",
                                                                    experiment_name= "resnet50V2")])

saving tensorboard log files to tensorflow hub/resnet50V2/20250121-151242
Epoch 1/5
24/24 [==============================] - 407s 17s/step - loss: 1.9322 - accuracy: 0.3693 - val_loss: 1.0559 - val_accuracy: 0.6836
Epoch 2/5
24/24 [==============================] - 395s 17s/step - loss: 0.8465 - accuracy: 0.7360 - val_loss: 0.7166 - val_accuracy: 0.7776
Epoch 3/5
24/24 [==============================] - 406s 17s/step - loss: 0.5868 - accuracy: 0.8280 - val_loss: 0.6428 - val_accuracy: 0.7940
Epoch 4/5
24/24 [==============================] - 386s 17s/step - loss: 0.4609 - accuracy: 0.8787 - val_loss: 0.5907 - val_accuracy: 0.8052
Epoch 5/5
24/24 [==============================] - 413s 18s/step - loss: 0.3536 - accuracy: 0.9320 - val_loss: 0.5511 - val_accuracy: 0.8172


In [1]:
!nvidia-smi

Tue Jan 21 15:09:29 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--